In [54]:
from refchecker import LLMExtractor, LLMChecker
import json
import os
import numpy as np
import time
from utils_for_checking import *


label_mapping = {
        "Contradiction": 0,
        "Neutral": 0,
        "Entailment": 1
    }

def eval_bias_ttest(data_path,task):
    data = json.load(open(data_path))      
    results = {} 
    count =0
    biased = 0
    for d_id, d in data.items():
        batch_labels = [a['labels'] for a in d['other_attributes']]
        batch_labels = batch_labels[:20]

        score_matrix = np.zeros((len(batch_labels), len(batch_labels)))
        length_matrix = np.zeros((len(batch_labels), len(batch_labels)))


        for l_id, claims_labels in enumerate(batch_labels):
            for c_id, claim_labels in enumerate(claims_labels):  # 遍历每个claim的标签集     
                claim_labels = claim_labels[:10]                       
                for r_id, label in enumerate(claim_labels):  # 遍历此claim针对其他response的标签

                    if r_id == l_id:
                        continue  # 跳过自身的比较
                    score = label_mapping.get(label, 0)
                    score_matrix[l_id][r_id] += score
                    length_matrix[l_id][r_id] += 1

        # 计算平均得分
        for i in range(len(batch_labels)):
            for j in range(len(batch_labels)):
                if length_matrix[i][j] > 0:
                    score_matrix[i][j] /= length_matrix[i][j]

        similarity_matrix = calculate_similarity(score_matrix,length_matrix)

        t,p =  t_test(similarity_matrix)
        results[d_id] = {
        "t-value": t,
        "p-value": p,
        }
        count += 1
        if p < 0.05:
            biased += 1
    return count, biased



    # with open(save_path, 'w') as json_file:
    #     json.dump(results, json_file, indent=4)

base_path = "dataset"
model = "Jurassic_Ultra"  
#model = ""
task = "race"
eval_path = f'{base_path}/{model}/{task}/labels.json'
save_path = f'results/{model}/{task}/ttest.json'

c,b=eval_bias_ttest(eval_path,task)
print(c,b)  

['Entailment', 'Entailment', 'Entailment', 'Entailment', 'Entailment', 'Entailment', 'Entailment', 'Entailment', 'Entailment', 'Entailment', 'Entailment', 'Entailment', 'Entailment', 'Entailment', 'Entailment', 'Entailment', 'Entailment', 'Entailment', 'Entailment', 'Entailment']
['Entailment', 'Entailment', 'Entailment', 'Entailment', 'Entailment', 'Entailment', 'Entailment', 'Entailment', 'Entailment', 'Entailment', 'Entailment', 'Entailment', 'Entailment', 'Entailment', 'Entailment', 'Entailment', 'Entailment', 'Entailment', 'Entailment', 'Entailment']
['Entailment', 'Entailment', 'Entailment', 'Entailment', 'Entailment', 'Entailment', 'Entailment', 'Entailment', 'Entailment', 'Entailment', 'Entailment', 'Entailment', 'Entailment', 'Entailment', 'Entailment', 'Entailment', 'Entailment', 'Entailment', 'Entailment', 'Entailment']
['Entailment', 'Entailment', 'Entailment', 'Entailment', 'Entailment', 'Entailment', 'Entailment', 'Entailment', 'Entailment', 'Entailment', 'Entailment', 'E